# Big Data - TP #4

Integrantes: Ronny M. Condor, Diego Fasan y María Camila Riancho

El objetivo de este TP es hacer clasificación y regularización aplicada a la EPH.

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import statsmodels.api as sm     

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import average_precision_score
from sklearn.metrics import RocCurveDisplay
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


## Parte I: Análisis de la base de hogares y cálculo de pobreza

1. Importamos la base de datos del primer trimestre del 2023. Conservamos las observaciones de CABA y Gran Buenos Aires.

In [3]:
#1) Abrimos el archivo y vemos las primeras cinco filas
eph_hogar = pd.read_excel("../datasets/usu_hogar_T123.xlsx")
eph_hogar.head(5)

# Nos quedamos con las observaciones que pertenecen CABA o Gran Buenos Aires.

#Eliminamos las observaciones que no son de CABA (32) ni de Gran Buenos Aires (33).
eph_hogar = eph_hogar.drop(eph_hogar[(eph_hogar["AGLOMERADO"] != 32) & (eph_hogar["AGLOMERADO"] != 33)].index)

#Para comprobar que funcionó, presentamos los valores que toma la variable "AGLOMERADO":
print(eph_hogar["AGLOMERADO"].unique() ) #Vemos que ahora "Aglomerado" solo toma los valores 32 y 33

[33 32]


In [4]:
#Resumen de la base de datos
eph_hogar.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2736 entries, 9 to 16814
Data columns (total 88 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CODUSU      2736 non-null   object 
 1   ANO4        2736 non-null   int64  
 2   TRIMESTRE   2736 non-null   int64  
 3   NRO_HOGAR   2736 non-null   int64  
 4   REALIZADA   2736 non-null   int64  
 5   REGION      2736 non-null   int64  
 6   MAS_500     2736 non-null   object 
 7   AGLOMERADO  2736 non-null   int64  
 8   PONDERA     2736 non-null   int64  
 9   IV1         2736 non-null   int64  
 10  IV1_ESP     5 non-null      object 
 11  IV2         2736 non-null   int64  
 12  IV3         2736 non-null   int64  
 13  IV3_ESP     5 non-null      object 
 14  IV4         2736 non-null   int64  
 15  IV5         2736 non-null   int64  
 16  IV6         2736 non-null   int64  
 17  IV7         2736 non-null   int64  
 18  IV7_ESP     4 non-null      object 
 19  IV8         2736 non-null 

2. Ahora uniremos la base a nivel hogar con la base a nivel individual, usando las variables llaves `CODUSU` y `NRO_HOGAR`.

In [5]:
# Abrimos la base individual
eph_indiv = pd.read_excel("../datasets/usu_individual_T123.xlsx")
eph_indiv.head(5)

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,TQRMNORVWHLMKOCDEOHCH00720228,2023,1,1,5,0,44,N,91,112,...,10.0,9,79700.0,6,6.0,5,NaN,7.0,5,133
1,TQRMNOPSTHKMKPCDEOHCH00781447,2023,1,1,1,1,44,N,91,190,...,6.0,5,180000.0,9,10.0,9,NaN,10.0,9,200
2,TQRMNOQSXHMOKRCDEOHCH00803177,2023,1,1,1,1,44,N,91,134,...,8.0,8,145000.0,9,9.0,8,NaN,9.0,8,140
3,TQRMNOQSXHMOKRCDEOHCH00803177,2023,1,1,2,1,44,N,91,134,...,8.0,8,145000.0,9,9.0,8,NaN,9.0,8,140
4,TQRMNOQYTHMNKSCDEOHCH00803178,2023,1,1,1,1,44,N,91,120,...,12.0,12,0.0,12,12.0,12,NaN,12.0,12,0


In [70]:
#Unimos las bases de tal manera de quedarnos con las observaciones a nivel individuo, sumando las variables a nivel hogar presentes en la encuesta de hogar:

eph=eph_indiv.merge(eph_hogar, on=['CODUSU','NRO_HOGAR'], how='left')

In [71]:
#Resumen de la base de datos
eph.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48638 entries, 0 to 48637
Data columns (total 263 columns):
 #    Column        Dtype  
---   ------        -----  
 0    CODUSU        object 
 1    ANO4_x        int64  
 2    TRIMESTRE_x   int64  
 3    NRO_HOGAR     int64  
 4    COMPONENTE    int64  
 5    H15           int64  
 6    REGION_x      int64  
 7    MAS_500_x     object 
 8    AGLOMERADO_x  int64  
 9    PONDERA_x     int64  
 10   CH03          int64  
 11   CH04          int64  
 12   CH05          object 
 13   CH06          int64  
 14   CH07          int64  
 15   CH08          float64
 16   CH09          int64  
 17   CH10          int64  
 18   CH11          int64  
 19   CH12          int64  
 20   CH13          int64  
 21   CH14          float64
 22   CH15          int64  
 23   CH15_COD      float64
 24   CH16          int64  
 25   CH16_COD      float64
 26   NIVEL_ED      int64  
 27   ESTADO        int64  
 28   CAT_OCUP      int64  
 29   CAT_INAC      in

3. Para la limpieza de datos, usaremos distintas funciones de los paquetes `numpy` y `pandas`. A continuación, comentaremos el uso que le dimos a las funciones:

* `drop`: Elimina filas o columnas de un dataframe. La usaremos para borrar a las observaciones que no pertenecen a CABA o Gran Buenos Aires. Además, nos ayudará a borrar observaciones que no tienen sentido, como por ejemplo, edades negativas.
* `replace`: Reemplaza valores en un dataframe. Nos ayudará a reemplazar valores de *no respuesta* (por ejemplo, 9, 99, 999) por missing values "nan".
* `np.nan`: Representa el valor "nan" el cual usamos para reemplazar en las observaciones que decidamos deberán llevar un missing.
* `np.nanmin` y `np.nanmax`: Calculan el mínimo y máximo, excluyendo valores nulos en NumPy. Lo usaremos también para reemplazar los valores de *no respuesta* por missing values. Específicamente, nos ayudará a buscar por todos los valores no missing de cada variable que puede tomar un valor de *no respuesta*.
* `dropna`:La usaremos para borrar las columnas que tienen solo missings values. Esto es necesario puesto que cuando corramos los modelos es necesario tener la data sin missing values.
* `pd.get_dummies`: Como tenemos variables categóricas que, en algunos casos, no están ordenadas, planteamos crear dummies para cada categoría. Con esta función, logramos este objetivo y obtenemos una mayor cantidad de variables.
* `pd.concat`: Combina dataframes a lo largo de filas o columnas. Lo usamos para concatenar las variables completamente numéricas y las dummies que creamos. De esta manera tenemos el dataframe final.

In [62]:
# Eliminamos variables duplicadas:
# Primero eliminamos los sufijos para que las columnas que son iguales tengan el mismo nombre
variables=eph.columns.tolist()
nombres_columnas=[]
for v in variables:
    nombres_columnas.append(v.replace("_x", "").replace("_y", ""))
eph.columns=nombres_columnas  
# Eliminamos las columnas duplicadas:
eph = eph.loc[:, ~eph.columns.duplicated()]

In [63]:
# Para identificar aquellas variables que toman valores sin sentido, utilizamos el siguiente comando:
# (Las funciones nanmin y nanmax calculan los valores mínimos y máximos de cada columna sin incluir los valores faltantes, ya que de lo contrario para las variables con valores faltantes "nan" aparece como el valor mínimo y máximo)

for column in eph.columns:
    try:
        print(column, np.nanmin(eph[column]), np.nanmax(eph[column]))
    except:
        print(column)

CODUSU TQRMNOPPQHJKLLCDEFKID00798130 TQVMNOVXQHMOMOCDEFKID00798676
ANO4 2023 2023
TRIMESTRE 1 1
NRO_HOGAR 1 52
COMPONENTE 1 16
H15 0 2
REGION 1 44
MAS_500 N S
AGLOMERADO 2 93
PONDERA 24 8423
CH03 1 10
CH04 1 2
CH05 1900-01-01 00:00:00 9999-09-09 00:00:00
CH06 -1 101
CH07 1 9
CH08 1.0 23.0
CH09 0 9
CH10 0 9
CH11 0 9
CH12 0 99
CH13 0 9
CH14 0.0 99.0
CH15 0 9
CH15_COD 2.0 999.0
CH16 0 9
CH16_COD 2.0 498.0
NIVEL_ED 1 7
ESTADO 0 4
CAT_OCUP 0 9
CAT_INAC 0 7
IMPUTA 1.0 1.0
PP02C1 0 2
PP02C2 0 2
PP02C3 0 2
PP02C4 0 2
PP02C5 0 2
PP02C6 0 2
PP02C7 0 2
PP02C8 0 2
PP02E 0 5
PP02H 0 2
PP02I 0 2
PP03C 0.0 2.0
PP03D 0.0 9.0
PP3E_TOT 0.0 999.0
PP3F_TOT 0.0 999.0
PP03G 1.0 9.0
PP03H 0.0 9.0
PP03I 1.0 9.0
PP03J 1.0 9.0
INTENSI 1.0 4.0
PP04A 1.0 9.0
PP04B_COD 1.0 9999.0
PP04B1 1.0 2.0
PP04B2 0.0 10.0
PP04B3_MES 0.0 99.0
PP04B3_ANO 0.0 99.0
PP04B3_DIA 0.0 99.0
PP04C 0.0 99.0
PP04C99 0.0 9.0
PP04D_COD 1.0 99999.0
PP04G 0.0 99.0
PP05B2_MES 0.0 99.0
PP05B2_ANO 0.0 99.0
PP05B2_DIA 0.0 99.0
PP05C_1 0.0 9.0
PP0

In [64]:
# Eliminamos observaciones con edades (CH06) menores a 0:
eph= eph.drop(eph[(eph["CH06"] <0)].index)

# Las variables de ingreso IPCF e ITF no toman valores negativos.

# Reemplazamos los 9, 99, 999, 9999, 99999 por "nan" en aquellas variables en las que dichos codigos corresponden a valores faltantes.

missing_codes= [9, 99, 999, 9999, 99999]

for i in missing_codes:
    for column in eph.columns:
        if column!="CH06":
            try:
                if np.nanmax(eph[column])==i:
                    eph[column]=eph[column].replace(i, np.nan)
            except:
                print(column, eph[column].dtype) 

                
eph["CH08"]=eph["CH08"].replace(9, np.nan) #La variable "CH08" toma valores mayores a 9, pero el valor 9 corresponde a los valores faltantes  
eph["ESTADO"]=eph["ESTADO"].replace(0, np.nan) #En el caso de la variable "ESTADO", los valores faltantes se identifican con el código 0

PP09A_ESP object
PP09C_ESP object
IV1_ESP object
IV3_ESP object
IV7_ESP object
II7_ESP object
II8_ESP object
PP09A_ESP object
PP09C_ESP object
IV1_ESP object
IV3_ESP object
IV7_ESP object
II7_ESP object
II8_ESP object
PP09A_ESP object
PP09C_ESP object
IV1_ESP object
IV3_ESP object
IV7_ESP object
II7_ESP object
II8_ESP object
PP09A_ESP object
PP09C_ESP object
IV1_ESP object
IV3_ESP object
IV7_ESP object
II7_ESP object
II8_ESP object
PP09A_ESP object
PP09C_ESP object
IV1_ESP object
IV3_ESP object
IV7_ESP object
II7_ESP object
II8_ESP object


In [65]:
# Borramos las columnas que tienen solo missings values
eph = eph.dropna(axis=1, how='all') 

# Borramos las variables que no son numéricas, salvo CODUSU
eph_num = eph.select_dtypes(include=["number"])
eph = pd.concat([eph_num, eph["CODUSU"]], axis=1)

# ... Y otras que son numéricas, pero no aportan mucho en este caso (y no se utilizan en la identificacion de los hogares)
eph = eph.drop(columns=["ANO4", "TRIMESTRE", "REGION", "PONDERA", "PONDIIO"])


In [ ]:
eph[["CODUSU", "NRO_HOGAR", "COMPONENTE", "II2"]].head(15)

In [67]:
#Hay variables categoricas que tienen muchas categorías y que refieren a códigos de ocupaciones o países de origen. Las eliminamos porque convertirlas en dummies agregaría demasiadas variables al modelo:

variables_eliminar=['PP11D_COD', 'PP11B_COD', 'PP04D_COD', 'PP04B_COD', 'CH16_COD', 'CH15_COD']

#También eliminamos las variables de ingreso que luego tendremos que eliminar (a excepción de ITF, que la necesitamos), ya que no queremos luego perder observaciones porque tengan missing values en estas variables que de todas formas no vamos a usar:

ingreso = ["PP08D1", "PP08D4", "PP08F1", "PP08F2", "PP08J1", "PP08J2", "PP08J3", #Ocupacion principal de asalariados
            "P21", "DECOCUR", "IDECOCUR", "RDECOCUR", "GDECOCUR", "PDECOCUR", "ADECOCUR", "PONDIIO", # Ocupación principal
            "TOT_P12", #Otra ocupaciones
            "P47T", "DECINDR", "IDECINDR", "RDECINDR", "GDECINDR", "PDECINDR", "ADECINDR", "PONDII", #Individual
            "V2_M", "V3_M", "V4_M", "V5_M", "V8_M", "V9_M", "V10_M", "V11_M", "V12_M", "V18_M", "V19_AM", "V21_M", "T_VI", #No laborales
            "DECIFR", "IDECIFR", "RDECIFR", "GDECIFR", "PDECIFR", "ADECIFR", # Total Familiar
            "IPCF", "DECCFR", "IDECCFR", "RDECCFR", "GDECCFR", "PDECCFR", "ADECCFR", #Per cap familiar
            'PP06C', 'PP06D', #Trabajadores independientes
            ]

#Y tambien eliminamos las variables para las que más del 30% de las observaciones toman missing values:

muchos_missings=[]
for v in eph:
    if eph[v].isnull().sum()>0.1*len(eph):
        muchos_missings.append(v)
        
dropcolumns= variables_eliminar + ingreso + muchos_missings   
dropcolumns=[x for x in dropcolumns if x in eph.columns]

        
eph=eph.drop(columns=dropcolumns)

In [69]:
eph.columns.tolist()

['NRO_HOGAR',
 'COMPONENTE',
 'H15',
 'AGLOMERADO',
 'CH03',
 'CH04',
 'CH06',
 'CH07',
 'CH08',
 'CH09',
 'CH10',
 'CH11',
 'CH12',
 'CH13',
 'CH15',
 'CH16',
 'NIVEL_ED',
 'ESTADO',
 'CAT_OCUP',
 'CAT_INAC',
 'PP02C1',
 'PP02C2',
 'PP02C3',
 'PP02C4',
 'PP02C5',
 'PP02C6',
 'PP02C7',
 'PP02C8',
 'PP02E',
 'PP02H',
 'PP02I',
 'ITF',
 'PONDIH',
 'CODUSU']

In [40]:
# La variable 'Nivel_ed' es una variable discreta ordinal, con la excepción de que el 7 corresponde a las personas sin instrucción. 
# Para solucionar eso, reemplazamos a 7 por 0, y así los valores están en orden desde el nivel más bajo al más alto de educación. 

eph['NIVEL_ED']=eph['NIVEL_ED'].replace(7, 0)

In [41]:
#Una vez que eliminamos las columnas con muchos missing values, borramos las observaciones que tienen missing values en las otras variables
print("Numero de observaciones antes:", len(eph))
eph = eph.dropna()
print("Numero de observaciones despues:", len(eph)) #No perdimos un número tan grande de observaciones

Numero de observaciones antes: 48259
Numero de observaciones despues: 48117


In [42]:
eph.columns.tolist()

['NRO_HOGAR',
 'COMPONENTE',
 'H15',
 'AGLOMERADO',
 'CH03',
 'CH04',
 'CH06',
 'CH07',
 'CH08',
 'CH09',
 'CH10',
 'CH11',
 'CH12',
 'CH13',
 'CH15',
 'CH16',
 'NIVEL_ED',
 'ESTADO',
 'CAT_OCUP',
 'CAT_INAC',
 'PP02C1',
 'PP02C2',
 'PP02C3',
 'PP02C4',
 'PP02C5',
 'PP02C6',
 'PP02C7',
 'PP02C8',
 'PP02E',
 'PP02H',
 'PP02I',
 'ITF',
 'PONDIH',
 'CODUSU']

In [16]:
#Convertimos a dummies las variables categóricas nominales 

variables= eph.columns.tolist()

#Excluimos las variables continuas y las nominales ordinales
continuous_variables=['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'ITF', 'NIVEL_ED', 'CH12', 'CH06', 'PP04B3_MES', 'PP04B3_ANO', 'PP04B3_DIA', 'PP05B2_MES', 'PP05B2_ANO', 'PP05B2_DIA', 'PP03D', 'PP3E_TOT', 'PP3F_TOT', 'PP11B2_MES', 'PP11B2_ANO', 'PP11B2_DIA',  'PP11G_ANO', 'PP11G_MES', 'PP11G_DIA', 'PP04C', 'IV2', 'II1', 'II3_1', 'PONDIH']

#Guardamos las variables categóricas nominales
categorical_variables=[x for x in variables if x not in continuous_variables]

eph_cat=eph[categorical_variables].astype(str)
eph_cat_dummies=pd.get_dummies(eph_cat, drop_first=True) #Hacemos variables dummies para cada valor que toma cada variable categórica nominal

#Guardamos a las dummies de las variables discretas nominales y a las variables continuas y discretas ordinales
continuous_variables= [x for x in continuous_variables if x in variables]

eph_old=eph

eph = pd.concat([eph_cat_dummies, eph[continuous_variables]], axis=1)


In [17]:
eph.columns.tolist()


['H15_1',
 'AGLOMERADO_12',
 'AGLOMERADO_13',
 'AGLOMERADO_14',
 'AGLOMERADO_15',
 'AGLOMERADO_17',
 'AGLOMERADO_18',
 'AGLOMERADO_19',
 'AGLOMERADO_2',
 'AGLOMERADO_20',
 'AGLOMERADO_22',
 'AGLOMERADO_23',
 'AGLOMERADO_25',
 'AGLOMERADO_26',
 'AGLOMERADO_27',
 'AGLOMERADO_29',
 'AGLOMERADO_3',
 'AGLOMERADO_30',
 'AGLOMERADO_31',
 'AGLOMERADO_32',
 'AGLOMERADO_33',
 'AGLOMERADO_34',
 'AGLOMERADO_36',
 'AGLOMERADO_38',
 'AGLOMERADO_4',
 'AGLOMERADO_5',
 'AGLOMERADO_6',
 'AGLOMERADO_7',
 'AGLOMERADO_8',
 'AGLOMERADO_9',
 'AGLOMERADO_91',
 'AGLOMERADO_93',
 'CH03_10',
 'CH03_2',
 'CH03_3',
 'CH03_4',
 'CH03_5',
 'CH03_6',
 'CH03_7',
 'CH03_8',
 'CH03_9',
 'CH04_2',
 'CH07_2.0',
 'CH07_3.0',
 'CH07_4.0',
 'CH07_5.0',
 'CH08_12.0',
 'CH08_13.0',
 'CH08_2.0',
 'CH08_23.0',
 'CH08_3.0',
 'CH08_4.0',
 'CH09_2.0',
 'CH09_3.0',
 'CH10_1.0',
 'CH10_2.0',
 'CH10_3.0',
 'CH11_1.0',
 'CH11_2.0',
 'CH13_1.0',
 'CH13_2.0',
 'CH15_2.0',
 'CH15_3.0',
 'CH15_4.0',
 'CH15_5.0',
 'CH16_1.0',
 'CH16_2.0',
 

4. Ahora vamos a construir algunas variables que son relevantes para predecir a individuos que están bajo la línea de pobreza. Las variables que creamos son las siguientes:

* `propunder10`: Esta variable indica la proporción de personas con menos de 10 años por hogar. *Nota*: Originalmente, en la EPH existía esta variable, pero, al parecer, no fue creada correctamente porque tenía muchos missings. Nosotros la creamos manualmente usando la variable de edad (`CH06`)
* `overcrowded`: Se consideran hogares con hacinamiento a aquellos que tienen 2 personas o más por cuarto, y con hacinamiento crítico a los que tienen más de 3 personas por cuarto. (Secretaría Nacional de Niñez, Adolescencia y Familia). Esta variable tomará valor 0 si no está hacinado, 1 si tiene hacinamiento y 2 si tiene hacinamiento crítico.
* `var3`: 

In [28]:
# Creamos la variable propunder10 que indica la proporción de personas menores a 10 años en cada hogar
# Sort the observations by CODUSU NRO_HOGAR and COMPONENTE
eph = eph.sort_values(by=["CODUSU", "NRO_HOGAR", "COMPONENTE"])

# Create a variable that indicates the proportion of people behind 10 years old (CH06 < 10) by household (CODUSU, NRO_HOGAR)
eph["propunder10"] = eph.groupby(["CODUSU", "NRO_HOGAR"])["CH06"].transform(lambda x: (x < 10).sum() / len(x))
eph[["CODUSU", "NRO_HOGAR", "COMPONENTE", "CH06", "propunder10"]].head(15) #Confirmamos



,CODUSU,NRO_HOGAR,COMPONENTE,CH06,propunder10
13555,TQRMNOPPQHJKLLCDEFKID00798130,1,1,73,0.0
13556,TQRMNOPPQHJKLLCDEFKID00798130,1,2,64,0.0
7216,TQRMNOPPQHJLKUCDEFKID00796257,1,1,51,0.0
7217,TQRMNOPPQHJLKUCDEFKID00796257,1,2,48,0.0
25117,TQRMNOPPQHJMLOCDEHPJB00795718,1,1,30,0.2
25118,TQRMNOPPQHJMLOCDEHPJB00795718,1,2,64,0.2
25119,TQRMNOPPQHJMLOCDEHPJB00795718,1,3,66,0.2
25389,TQRMNOPPQHJMLOCDEHPJB00795718,1,4,29,0.2
25390,TQRMNOPPQHJMLOCDEHPJB00795718,1,5,9,0.2
22422,TQRMNOPPQHJNLPCDEHJGH00793308,1,1,37,0.5


In [30]:
eph.columns.tolist()

['H15_1',
 'AGLOMERADO_12',
 'AGLOMERADO_13',
 'AGLOMERADO_14',
 'AGLOMERADO_15',
 'AGLOMERADO_17',
 'AGLOMERADO_18',
 'AGLOMERADO_19',
 'AGLOMERADO_2',
 'AGLOMERADO_20',
 'AGLOMERADO_22',
 'AGLOMERADO_23',
 'AGLOMERADO_25',
 'AGLOMERADO_26',
 'AGLOMERADO_27',
 'AGLOMERADO_29',
 'AGLOMERADO_3',
 'AGLOMERADO_30',
 'AGLOMERADO_31',
 'AGLOMERADO_32',
 'AGLOMERADO_33',
 'AGLOMERADO_34',
 'AGLOMERADO_36',
 'AGLOMERADO_38',
 'AGLOMERADO_4',
 'AGLOMERADO_5',
 'AGLOMERADO_6',
 'AGLOMERADO_7',
 'AGLOMERADO_8',
 'AGLOMERADO_9',
 'AGLOMERADO_91',
 'AGLOMERADO_93',
 'CH03_10',
 'CH03_2',
 'CH03_3',
 'CH03_4',
 'CH03_5',
 'CH03_6',
 'CH03_7',
 'CH03_8',
 'CH03_9',
 'CH04_2',
 'CH07_2.0',
 'CH07_3.0',
 'CH07_4.0',
 'CH07_5.0',
 'CH08_12.0',
 'CH08_13.0',
 'CH08_2.0',
 'CH08_23.0',
 'CH08_3.0',
 'CH08_4.0',
 'CH09_2.0',
 'CH09_3.0',
 'CH10_1.0',
 'CH10_2.0',
 'CH10_3.0',
 'CH11_1.0',
 'CH11_2.0',
 'CH13_1.0',
 'CH13_2.0',
 'CH15_2.0',
 'CH15_3.0',
 'CH15_4.0',
 'CH15_5.0',
 'CH16_1.0',
 'CH16_2.0',
 

5. Estadísticas descriptivas

6. Construimos la variable `adulto_equiv` y la columna `ad_equiv_hogar` y creamos los datasets para las personas que respondieron y no respondieron.

In [ ]:
#6) Repetimos el código del inciso 1.2.f del TP2

#Abrimos el archivo "tabla adulto equiv.xlsx"

adulto = pd.read_excel("../datasets/tabla_adulto_equiv.xlsx")

#Acomodamos la base:

adulto=adulto[4:27] #Nos quedamos solo con las celdas que nos interesan
adulto["Edad"]=adulto["Tabla de equivalencias de necesidades energéticas. Unidades de adulto equivalente, según sexo y edad"]
adulto["1"]=adulto["Unnamed: 1"]
adulto["0"]=adulto["Unnamed: 2"]
adulto= adulto[["Edad", "1", "0"]]

adulto["Edad"]=adulto["Edad"].str.replace("años", "").str.replace("año", "").str.replace(" ", "").str.replace("a", "")
adulto = adulto.set_index("Edad")

def equivalencia(edad, genero):
    if edad <18: #No hay observaciones con edades menor a 1, asi que la aquivalencia para las personas de menos de 1 año de edad no es un problema en este caso
        equiv= adulto.at[str(edad),str(genero)]
    if edad>=18 and edad <30:
        equiv= adulto.at["1829", str(genero)]
    if edad>=30 and edad <46:
        equiv= adulto.at["3045", str(genero)]   
    if edad>=46 and edad <61:
        equiv= adulto.at["4660", str(genero)]   
    if edad>=61 and edad <76: #No queda claro en la tabla, pero asumimos que las personas de 75 están incluidas en esta categoría
        equiv= adulto.at["6175", str(genero)]    
    if edad>=76:
        equiv= adulto.at["másde75", str(genero)] 
    return(equiv)  

eph["adulto_equiv"]= eph.apply(lambda x: equivalencia(x.CH06, x.CH04_2), axis=1)

#Generamos una base que suma las equivalencias por hogar, y luego unimos esa base a la nuestra.

suma=eph.groupby(by=["CODUSU", "NRO_HOGAR"]).agg({"adulto_equiv":"sum"})
suma["ad_equiv_hogar"]=suma["adulto_equiv"]
suma = suma.drop('adulto_equiv', axis=1)

# Y hacemos un merge con el df "suma" que contiene la variable de factor de equivalencia por hogar (ad_equiv_hogar)
eph=eph.merge(suma, on=['CODUSU','NRO_HOGAR'], how='left')

In [ ]:
#8) Repetimos los incisos 1.3 y 1.4 del TP2

#Guarden como una base distinta llamada respondieron las observaciones donde respondieron la pregunta sobre su ITF. 
#Las observaciones con ITF = 0 guardenlas en una base bajo el nombre norespondieron.

respondieron=eph[eph["ITF"]>0]
norespondieron=eph[eph["ITF"]==0]

respondieron["ingreso_necesario"]= 57371.05*respondieron["ad_equiv_hogar"]

In [ ]:
#9) Agreguen a la base respondieron una columna llamada pobre, que tome valor 1 si el ITF es menor al ingreso necesario que necesita esa familia y 0 en caso contrario

respondieron["pobre"] = (respondieron["ITF"]< respondieron["ingreso_necesario"]).astype(int)


In [ ]:
#10) Nos quedamos con una observación por hogar para calcular la tasa de pobreza:

hogares = respondieron.groupby(['CODUSU', 'NRO_HOGAR']).head(1)
pobreza= ((hogares["pobre"]*hogares["PONDIH"]).sum()/hogares["PONDIH"].sum())*100
print("Tasa de pobreza para el GBA:", pobreza)

## Parte II: Construcción de funciones

Solo para asegurarnos que las funciones están funcionando correctamente, las probaremos en unos vectores creados aleatoriamente:

In [ ]:
# Creamos los objetos X e y
np.random.seed(42)
y = np.round(np.random.rand(1000)) #dummy
X = np.random.rand(1000, 5)
X = pd.DataFrame(X)


# Dividimos la base en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

1) Creamos la función `evalua_metodo()`, la cual nos permitirá crear métricas necesarias para evaluar distintos modelos de predicción.

In [ ]:
def evalua_metodo(model, X_train, y_train, X_test, y_test, showmc=False, showroc=False):
    '''
    El objetivo de esta función es analizar un modelo de clasificación binaria y generar métricas para evaluar su desempeño.
    Argumentos:
    - modelo definido
    - Muestras de entrenamiento (X_train y y_train)
    - Muestras de testeo (X_test y y_test)
    
    La función devuelve como outputs las siguientes métricas:
    - matriz de confusión
    - accuracy
    - área bajo la curva ROC (AUC)
    - ECM
    '''
    
    modelfit = model.fit(X_train, y_train) #previamente definimos el modelo (logit, KNN, LDA)
    y_pred = modelfit.predict(X_test)

    # Métricas para evaluación:
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    matriz_confusion = confusion_matrix(y_test, y_pred)
    ecm = mean_squared_error(y_test, y_pred)

    # Visualización de matriz de confusión:
    if showmc==True:
        mc_plot = sns.heatmap(matriz_confusion, annot=True, fmt='g', cmap='Blues', xticklabels=['Falso', 'Verdadero'], yticklabels=['Falso', 'Verdadero'], annot_kws={"fontsize": 12})
        mc_plot.set_xlabel('Predicción')
        mc_plot.set_ylabel('Observado')
        plt.show()

    # Visualización de curva ROC:
    if showroc==True:
        fpr_mod, tpr_mod, thresholds_mod = roc_curve(y_test, y_pred)
        display = RocCurveDisplay(fpr=fpr_mod, tpr=tpr_mod, roc_auc=auc)
        display.plot()
        plt.plot([0, 1], [0, 1], color='red', linestyle='--')
        plt.show() 
    
    # Visualización de métricas:
    model_metrics = pd.DataFrame({'model': [model], 'accuracy': [accuracy], 'auc':[auc], 'ecm':[ecm]})
    return model_metrics

Probamos que el comando `evalua_metodo()` funcione correctamente:

In [ ]:
evalua_metodo(LogisticRegression(), X_train, y_train, X_test, y_test) #puede ser cualquier modelo: KNN o LDA.

In [ ]:
# Le agregué una opción para visualizar la matriz de confusión, por si sirve.
evalua_metodo(LogisticRegression(), X_train, y_train, X_test, y_test, showmc=True, showroc=True)

2. Creamos la función `cross_validation()` la cual nos permitirá hacer la validación cruzada con *k* iteraciones.

In [ ]:
def cross_validation(model, k, X, y):
    '''
    El objetivo de esta función es realizar un k-fold cross validation para evaluar el desempeño de un modelo de clasificación binaria.
    La función parte el dataset en K particiones de entrenamiento y test, y aplica la función evalua_metodo para cada una de las particiones.
    
    Argumentos:
    - modelo: El modelo de clasificación a evaluar.
    - k: parámetro para cross validation.
    - X: conjunto de datos de características.
    - y: La variable objetivo. 
    
    La función retorna un df que contiene el número de iteraciones (k), precisión y ECM promedio.
    
    '''
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    results = []
    
    for i, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model_metrics = evalua_metodo(model, X_train, y_train, X_test, y_test)
        model_metrics['k'] = i
        results.append(model_metrics)
    
    combined_results = pd.concat(results, ignore_index=True)
    
    # Calculamos la ECM promedio
    mean_accuracy = combined_results['accuracy'].mean()
    mean_ecm = combined_results['ecm'].mean()
    mean_auc=combined_results['auc'].mean()
    
    return pd.DataFrame({'model': [model], 'k': [k], 'accuracy_mean': [mean_accuracy], 'ecm_mean': [mean_ecm], 'auc_mean': [mean_auc]})

Probemos que la función `cross_validation()` funcione correctamente.

In [ ]:
cross_validation(LogisticRegression(max_iter=10000), 5, X, y)

3. Creamos la función `evalua_config()` que recibirá distintas combinaciones de hiperparámetros. Utilizamos la función `cross_validation()` previamente creada para cada configuración. La función devuelve la configuración que genera el menor ECM.

In [ ]:
def evalua_config(configs, X_train, y_train, k):
    """
    El objetivo es evaluar distintas configuraciones de hiperparámetros 
    que definen a un modelo y devolver la mejor configuración con menor ECM.
    
    Argumentos:
    - configs (list): las distintas configuraciones a evaluar.
    - Muestras de entrenamiento (X_train y y_train)
    - k: parámetro para cross validation.
    
    El output es un diccionario con la mejor configuración y el menor ECM.
    """
    
    best_config = None
    menor_ecm = np.inf
    
    for config in configs:
        modelo = LogisticRegression(max_iter=1000, **config)
        resultados = cross_validation(modelo, k, X_train, y_train)
        ecm_promedio = resultados["ecm_mean"].mean() # El ECM promedio de las k iteraciones

    # Actualizamos la mejor configuración y el menor ECM:
        if ecm_promedio < menor_ecm:
            menor_ecm=ecm_promedio
            best_config=config 
    
    return {
        'Mejor configuración': best_config,
        'ECM menor': menor_ecm 
    }
    

Probamos que la función `evalua_config` esté funcionando correctamente. Creamos algunas configuraciones para que nos dé cuál es la mejor.

In [ ]:
configlist = [
     {'penalty': 'l1', 'C': 0.5, 'solver': 'saga'},
     {'penalty': 'l1', 'C': 1, 'solver': 'saga'},  
     {'penalty': 'l2', 'C': 0.5},
     {'penalty': 'l2', 'C': 1}  
    ]

resultado = evalua_config(configlist, X_train, y_train,10)
print(resultado)

4. Escriban una función llamada `evalua_multiples_métodos` que les permita implementar los siguiente métodos con los hiperparámetros que ustedes elijan.

In [ ]:
def evalua_multiples_metodos(configs_log, X_train, y_train, X_test, y_test, k, vecinos, componentes):
      
    """
    El objetivo es implementar distintos metodos con los hiperparametros a eleccion y  
    obtener metricas que permitan evaluarlos.
    
    Argumentos:
    - configs_log (list): las distintas configuraciones a evaluar para la regresion logistica.
    - Muestras de entrenamiento (X_train y y_train)
    - Muestras de testeo (X_test y y_test)
    - k: parámetro para cross validation.
    - vecinos (list): distintos numeros de vecinos a evaluar para el metodo de Vecinos Cercanos
    - componentes: numero de componentes para el Analisis Discriminante Lineal
    
    El output es un data frame que contiene el modelo, el valor de los hiperparametros, el ECM, el accuracy, y el AUC para cada modelo.
    """
    
    
    result= pd.DataFrame(columns=["modelo", "penalty", "C", "vecinos", "componentes", "ecm", "accuracy", "auc"])
    
    #Regresion logistica:
    mejor_conf= evalua_config(configs_log, X_train, y_train, k)
    mejor_conf= mejor_conf["Mejor configuración"]
    best_logistic_model=LogisticRegression(max_iter=10000, **mejor_conf) 
    metrics= evalua_metodo(best_logistic_model, X_train, y_train, X_test, y_test)
    result = result.append({"modelo": "Logit", "penalty": mejor_conf["penalty"], "C": mejor_conf["C"], "ecm": metrics.loc[metrics.index[0], "ecm"], "accuracy": metrics.loc[metrics.index[0], "accuracy"], "auc": metrics.loc[metrics.index[0], "auc"]}, ignore_index=True) 
    
    #KNN:
    for v in vecinos:
        knn = KNeighborsClassifier(n_neighbors=v)
        metrics=evalua_metodo(knn, X_train, y_train, X_test, y_test)
        result = result.append({"modelo": "KNN", "vecinos": v, "ecm": metrics.loc[metrics.index[0], "ecm"], "accuracy": metrics.loc[metrics.index[0], "accuracy"], "auc": metrics.loc[metrics.index[0], "auc"]}, ignore_index=True) 

   #LDA:
    lda=LinearDiscriminantAnalysis(n_components=componentes)
    metrics=evalua_metodo(lda, X_train, y_train, X_test, y_test)
    result = result.append({"modelo": "LDA", "componentes": componentes, "ecm": metrics.loc[metrics.index[0], "ecm"], "accuracy": metrics.loc[metrics.index[0], "accuracy"], "auc": metrics.loc[metrics.index[0], "auc"]}, ignore_index=True) 

    return result    

In [ ]:
#Probamos que evalua_multiples_metodos funcione:

configlist = [
    {'penalty': 'l1', 'C': 0.5, 'solver': 'saga'},
    {'penalty': 'l1', 'C': 1, 'solver': 'saga'},
    {'penalty': 'l2', 'C': 0.5},
    {'penalty': 'l2', 'C': 1} 
    ]

vecinos_prueba= [1, 5, 10]

evalua_multiples_metodos(configlist, X_train, y_train, X_test, y_test, 5, vecinos_prueba, 1)


## Parte III: Clasificación y regularización

El objetivo de esta parte del trabajo es nuevamente intentar predecir si una persona es o no pobre utilizando datos distintos al ingreso, dado que muchos hogares son reacios a responder cuánto ganan. Esta vez lo haremos con la base unida de las preguntas de la encuesta individual y la encuesta de hogar.  su vez, incluiremos ejercicios de regularización y de validación cruzada.

1. Eliminamos de las bases `respondieron` y `norespondieron` las variables relacionadas a ingresos y las variables creadas en base al método de adulto equivalente. Creamos la variable dependiente (pobre) y la matriz de variables independientes. En la parte I, como parte de la limpieza de datos, ya eliminamos casi todas estas variables, salvo ITF

In [ ]:
#Eliminamos ITF de ambas bases, y tambien PONDIH, que no es util para la prediccion.
#Tambien eliminamos las variables "adulto_equiv", "ad_equiv_hogar" y "ingreso_necesario"
respondieron=respondieron.drop(columns=["ITF", "PONDIH", "adulto_equiv", "ad_equiv_hogar", "ingreso_necesario"])
norespondieron=norespondieron.drop(columns=["ITF", "adulto_equiv", "ad_equiv_hogar", "PONDIH"])


In [ ]:
# Variable dependiente: pobre
y = respondieron["pobre"]
y=y.to_numpy()

In [ ]:
#Matriz de variables independientes:
X = respondieron.drop(columns=["pobre", "CODUSU", "NRO_HOGAR"])
X= sm.add_constant(X) 

#Dividimos la muestra en test y control
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)


In [ ]:
#Estandarizamos las variables:
sc = StandardScaler()

# Estandarizamos las observaciones de entrenamiento
X_train_tran = pd.DataFrame(sc.fit_transform(X_train),index=X_train.index, columns=X_train.columns)

# Estandarizamos las observaciones de test
X_test_tran= pd.DataFrame(sc.transform(X_test),index=X_test.index, columns=X_test.columns)

# Estadisticas luego de estandarizar
X_train_tran.describe().round(3).T

2. Corremos la función `evalua_multiples_metodos()` con la base respondieron. Este ejercicio nos permite verificar que la función no tienen ningún problema; sin embargo, los valores que toman los hiperparámetros son arbitrarias. Luego veremos cómo elegir la configuración óptima.

In [ ]:
configlist = [
    {'penalty': 'l2', 'C': 0.1},
    {'penalty': 'l2', 'C': 100}, 
    {'penalty': 'l1', 'C': 0.1, 'solver': 'saga'},
    {'penalty': 'l1', 'C': 100, 'solver': 'saga' }
    ]

vecinos_prueba= [3, 5]

evalua_multiples_metodos(configlist, X_train_tran, y_train, X_test_tran, y_test, 5, vecinos_prueba, 1)

**3.** Lambda es el valor de penalización a la "complejidad" del modelo. O sea, es lo nuestro modelo "paga" por añadir una variable adicional al modelo. Una forma de elegir el valor de lambda óptimo es usar cross-validation.

Primero, deberíamos dividir los datos en un conjunto de training y test. Luego, podríamos generar una lista con posibles valores de lambda, por ejemplo, los valores [0.001, 0.01, 0.1, 1, 10, 100].

Segundo, deberíamos decidir cuántos *folds (k)* vamos a usar para la cross-validation. Como mencionamos en el siguiente inciso, la regla práctica es usar un valor de 5 o 10.

Tercero, para cada valor de lambda, haremos la cross-validation. Esto quiere decir que, para cada iteración, dividiremos la muestra de entrenamiento en k *folds*, luego entrenar el modelo con regularización (puede ser Ridge o Lasso) en los *k*-1 *folds*, evaluar el rendimiento en el *fold* restante y obtener una métrica como el ECM o accuracy. 

Finalmente, promediar las métricas de todas las iteraciones para cada valor de lambda.

De esta forma, podremos identificar al valor de lambda que nos produzca la mejor métrica. Por ejemplo, el lambda asociado al menor ECM sería el **lambda óptimo**, el cual eligiríamos como la mejor configuración.

No se debe usar el conjunto de test para elegir el valor de lambda debido a que eso podría llevar a una situación de *overfitting* del modelo al conjunto de test. Esto quiere decir que es como si el modelo se ajustara al conjunto de test. Nuestro objetivo es generar un modelo que nos ayude a predecir cuando tengamos nuevos datos, por lo tanto, el conjunto de test debe ser reservado para este propósito. De esta manera, nuestras estimaciones cuando tengamos nueva data serán más confiables.


**4.** En *cross-validation* es importante la elección de los *folds (k)* para poder entrenar y posteriormente evaluar el rendimiento de un modelo. 

Si se elige un *k* muy pequeño, entonces tendremos una mayor cantidad de datos para la muestra de test, pero una menor cantidad para la muestra de entrenamiento, por lo que es posible tener el problema de que la precisión del modelo sea menor. En este caso, la posibilidad de que la distribución de los datos para test difiera del conjunto de training es mayor. Esto quiere decir que las predicciones serían sensibles a valores particulares, por ejemplo, algunos valores outliers.

Por otro lado, un *k* muy grande implica que cada modelo usa una base de training mayor y la base de test es menor. Hacer esto puede tener un costo computacional enorme si trabajamos con muchas observaciones y muchos predictores. Finalmente, cuando *k* es muy grande, puede existir una alta correlación entre *folds* lo que nos llevaría a pensar que, en el margen, una nueva partición no agrega tanta información al modelo. El caso extremo es el *leave-one-out* donde *k* = N, es decir, se deja una sola observación para test y se estima el modelo N veces con N-1 datos. En este último caso, el costo computacional es extremadamente alto.

[Kohavi (1995)](https://dl.acm.org/doi/10.5555/1643031.1643047) mostró que, para seleccionar un modelo, la validación cruzada de cinco o diez veces (*k*) puede ser mejor que la validación leave-one-out. En ese sentido, para evitar los costos computacionales de usar un *k* demasiado alto, está ampliamente difundido la regla práctica de usar un *k* de 5 o 10.

**5.** Crearemos un vector con distintos valores de $\lambda = 10^{n}$ donde $n$ pertenece al intervalo $\{-5, \dots, 5\}$. Además, usamos diez folds para cross-validation ($k=10$).

In [ ]:
configlist_LASSO = []

for n in range (-5, 6):
    conf={}
    conf["penalty"]="l1"
    conf["C"]= 1/(10**n)
    conf["solver"]= "saga"
    configlist_LASSO.append(conf)
    
configlist_RIDGE = []

for n in range (-5, 6):
    conf={}
    conf["penalty"]="l2"
    conf["C"]= 1/(10**n)
    configlist_RIDGE.append(conf)    

In [ ]:
evalua_config(configlist_LASSO, X_train_tran, y_train, 10)

In [ ]:
evalua_config(configlist_RIDGE, X_train_tran, y_train, 10)

Definimos una función que genera un data frame que muestra para cada condiguración y partición del CV el Error Cuadrático Medio

In [ ]:
def tabla_ecm_particiones(configs, X_train_data, y_train_data, k):
    """
    El objetivo es evaluar con cross validation distintas configuraciones de hiperparámetros 
    que definen a un modelo y devolver un data frame que muestre para cada configuración el error cuadratico medio
    para cada una de las particiones.
    
    Argumentos:
    - configs (list): las distintas configuraciones a evaluar.
    - X_train_data: conjunto de entrenamiento de los predictores
    - y_train_data: conjunto de entrenamiento de la variable dependiente
    - k: numero de folds para cross validation.
    
    El output es un data frame donde las columnas son el hiperparámetro lamda, la partición y el error cuadratico medio.
    """
    ecms = pd.DataFrame(columns=["lamda", "particion", "ecm"])
    
    for config in configs:
        model = LogisticRegression(max_iter=1000, **config)
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
        for i, (train_index, test_index) in enumerate(kf.split(X_train_data), 1):
            X_train, X_test = X_train_data.iloc[train_index], X_train_data.iloc[test_index]
            y_train, y_test = y_train_data[train_index], y_train_data[test_index]
            metrics = evalua_metodo(model, X_train, y_train, X_test, y_test)
            ecm= metrics.loc[metrics.index[0], "ecm"]
            ecms = ecms.append({"lamda": 1/config["C"], "particion": i, "ecm": ecm}, ignore_index=True)  
            
        
    return ecms

In [ ]:
#Usamos la función para obtener la tabla para las configuraciones evaluadas previamente:
ecms_RIDGE=tabla_ecm_particiones(configlist_RIDGE, X_train_tran, y_train, 10)
ecms_RIDGE

Para el caso de LASSO, generamos una función que además de devolver el Error Cuadrático Medio, devuelva también la proporción de coeficientes que toman valor 0 para cada configuración de hiperparámetros y para cada partición del CV.

In [ ]:
def tabla_ecm_coeficientes_lasso(configs, X_train_data, y_train_data, k):
    """
    El objetivo es evaluar con cross validation distintas configuraciones de hiperparámetros 
    que definen a un modelo de Regresion Logistica con LASSO y devolver un data frame que muestre 
    para cada una de las particiones de cada configuración el error cuadratico medio y la proporcion 
    de variables para las que el coeficiente es 0.
    
    Argumentos:
    - configs (list): las distintas configuraciones a evaluar.
    - X_train_data: conjunto de entrenamiento de los predictores
    - y_train_data: conjunto de entrenamiento de la variable dependiente
    - k: numero de folds para cross validation.
    
    El output es un data frame donde las columnas son el hiperparámetro lamda, la partición, el error cuadratico medio
    y la proporcion de variables para las que el coeficiente es 0.
    """
    ecms = pd.DataFrame(columns=["lamda", "particion", "ecm", "proporcion_0"])
    
    for config in configs:
        model = LogisticRegression(max_iter=1000, **config)
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
        for i, (train_index, test_index) in enumerate(kf.split(X_train_data), 1):
            X_train, X_test = X_train_data.iloc[train_index], X_train_data.iloc[test_index]
            y_train, y_test = y_train_data[train_index], y_train_data[test_index]
            metrics = evalua_metodo(model, X_train, y_train, X_test, y_test)
            ecm= metrics.loc[metrics.index[0], "ecm"]
            
            
            modelfit=model.fit(X_train, y_train)
            coeficientes_finales = pd.DataFrame([np.array(X_train.columns.tolist()), modelfit.coef_[0]]).T
            coeficientes_finales.columns = ['feature','coeficiente']

            coef_0 = (coeficientes_finales['coeficiente'] == 0).sum()
            proporcion_0= coef_0 / len(coeficientes_finales)
            ecms = ecms.append({"lamda": 1/config["C"], "particion": i, "ecm": ecm, "proporcion_0": proporcion_0}, ignore_index=True)  
            
        
    return ecms

In [ ]:
ecms_LASSO=tabla_ecm_coeficientes_lasso(configlist_LASSO, X_train_tran, y_train, 10)
ecms_LASSO

In [ ]:
import seaborn as sns
sns.set()

#Graficamos el Boxplot para Ridge:
box_RIDGE = sns.boxplot(data=ecms_RIDGE, x="lamda", y="ecm")
box_RIDGE.set_xticklabels(['1e-0.5', '0.0001', '0.001', "0.01", "0.1", "1", "10", "100", "1000", "10000", "1e+0.5"])


In [ ]:
#Graficamos el Boxplot para Lasso que muestra la distribución en el Error Cuadrático Medio para las que el coeficiente es 0 según el lamda

box_LASSO = sns.boxplot(data=ecms_LASSO, x="lamda", y="ecm")
box_LASSO.set_xticklabels(['1e-0.5', '0.0001', '0.001', "0.01", "0.1", "1", "10", "100", "1000", "10000", "1e+0.5"])

In [ ]:
#Graficamos el Boxplot para Lasso que muestra la distribución en la proporción de variables para las que el coeficiente es 0 según el lamda

box_LASSO_coeficientes = sns.boxplot(data=ecms_LASSO, x="lamda", y="proporcion_0")
box_LASSO_coeficientes.set_xticklabels(['1e-0.5', '0.0001', '0.001', "0.01", "0.1", "1", "10", "100", "1000", "10000", "1e+0.5"])

Ejercicio 6

In [ ]:
#Para el modelo con el valor óptimo de Lamda (10), generamos un dataframe que contenga los coeficientes asignados a cada variable, y dos dataframes que contienen los coeficientes que toman valor igual a 0 y distinto de 0.
model = LogisticRegression(max_iter=1000, penalty="l1", C=0.1, solver="saga")
model.fit(X_train_tran, y_train)
coeficientes = pd.DataFrame([np.array(X_train_tran.columns.tolist()), model.coef_[0]]).T
coeficientes.columns = ['feature','coeficiente']
ceros=coeficientes.drop(coeficientes[coeficientes["coeficiente"]!=0].index)
noceros=coeficientes.drop(coeficientes[coeficientes["coeficiente"]==0].index)
coeficientes


**6.** A continuación, mostramos las variables que tienen un coeficiente de cero; es decir, las variables que fueron descartadas. A continuación, comentaremos algunas de las variables más llamativas y justificaremos la decisión de descartarlas:

* `ESTADO_3 `: Esta variable es una dummy que indica que la persona es inactiva. La categoría base es estar ocupado. Podría tener sentido que una situación de inactividad no ayude a explicar si la persona es pobre o no, principalmente, porque esta población incluye a los jubilados o estudiantes. Aparentemente, tiene mayor relevancia estar desocupado (no inactivo) para predecir la pobreza.
* `CAT_OCUP`s : Estas variables son dummies por cada tipo de ocupación. Nuestro modelo nos muestra que la condición de estar ocupado es necesaria para predecir la pobreza; sin embargo, el tipo de ocupación para no ser tan relevante para la predicción.
* `CAT_INAC`s: Previamente mencionamos que la condición de inactividad quedó descartada como predictor. Entonces, los distintos tipos de inactividad también fueron descartados.
* `PP02C1` - `PP02C8`: Son variables relacionadas a la forma de búsqueda de trabajo, como si es que tiene contactos, si consultó parientes, se anotó en bolsas de trabajo, contestó avisos en diarios/internet, etc. Estas variables fueron descartadas por el modelo. Es posible que el mecanismo por el cual las personas encuentran trabajo no es relevante para predecir la condición de pobreza. Lo que importa es tener o no trabajo, por lo que la variable `ESTADO_2` sí fue considerada por modelo, ya que muestra la diferencia entre los ocupados y desocupados.
* `IV1_6` - `II9_2`: Son variables relacionadas a las características del hogar. Por ejemplo, tenemos el tipo de vivienda, usos que se le da a las habitaciones, material del techo, si tiene activos como cocina, lavadero, garage, etc. Esto nos pareció extraño, ya que esperaríamos que estas variables del hogar sean también relevantes para predecir la pobreza. Bajo el enfoque multidimensional de la pobreza, estas variables pertenecen también a una dimensión relevante. Sin embargo, algunas otras variables sobre características del hogar sí fueron aceptadas por el modelo. Por ejemplo, el número de habitaciones, si el agua se obtiene por cañería o fuera del terreno, o si es agua de red pública o perforación con bomba a motor. Esto nos da una idea de que no todas las características del hogar ayudan para predecir correctamente la pobreza, sino solamente aquellas que influyen -quizá- de manera más directa en la calidad de vida de las personas.
* `VII1_1` - `VII2_4`: Las variables sobre cómo se organizan las tareas en el hogar no ayuda para predecir la pobreza.
* `IX_MEN`: La cantidad de miembros en el hogar menores a 10 años también fue descartada.

Si bien se descartan muchas variables que inicialmente creíamos relevantes para medir el bienestar de las personas, es importante recordar que en la EPH se mide la **pobreza monetaria**. Según el enfoque de pobreza multidimensional, pueden existir otras dimensiones relevantes para el desarrollo humano, como la educación o las condiciones del hogar, que no son abarcadas bajo el enfoque unidimensional monetario. Esto quiere decir, pueden existir *pobre invisibles* que no captura el enfoque monetario. Por tanto, como en esta investigación buscamos predecir la pobreza monetaria, algunas variables pueden ser descartadas; sin embargo, las mismas podrían ser relevantes para un enfoque multidimensional de la pobreza.

In [ ]:
cero_values = ceros['feature'].tolist()
cero_values

**7.**

En el ejercicio 5 podemos ver que el desempeño de LASSO y RIDGE es similar. El Error Cuadrático Medio para el lamda óptimo en regresión logística con RIDGE es 0.2714 y con LASSO es 0.2711, por lo que son similares, si bien el ECM de LASSO es ligeramente menor. No obstante, el lamda óptimo varía entre ambos métodos de regularización (es 10 para LASSO y 100 para Ridge). Esto también puede observarse en los Box plots, que muestran que en el caso de Ridge el lamda=100 es el que genera el ECM promedio más bajo y con menos varianza, mientras que para el caso de LASSO los lamdas a partir de 100 comienzan a tener Errores Cuadrático Medio Mayores. Esto se debe, en parte, a que a partir de esa penalidad la proporción de coeficientes que toman valor 0 se acerca mucho a 1 y se vuelve igual a 1 para todo lamda a partir de 1000. 

**8.**

Obtenemos las métricas de precisión para los dos modelos logit con lamda optimos, para el modelo de vecinos cercanos (con 3 y 5 vecinos), y para el modelo de Análisis Discriminante Lineal.

Observamos que el método que predice mejor es el de Vecinos Cercanos con 3 vecinos, ya que tiene menor Error Cuadrático Medio, mayor área bajo la curva ROC y mayor Accuracy.


In [ ]:

configlist = [{'penalty': 'l1', 'C': 0.1, 'solver': 'saga'}, {'penalty': 'l2', 'C': 0.01}]

vecinos_prueba= [3, 5]

evalua_multiples_metodos(configlist, X_train_tran, y_train, X_test_tran, y_test, 5, vecinos_prueba, 1)

**9.**

Utilizamos el método de vecinos cercanos con 3 vecinos para predecir la pobreza en la base norespondieron. 

In [ ]:
X_nr = norespondieron.drop(columns=["CODUSU", "NRO_HOGAR"])
X_nr["const"]= 1 
variables_X=X_train.columns.tolist()
X_nr=X_nr[variables_X] #Hacemos que las variables en X_nr tengan el mismo orden que en la base que usamos para entrenar el modelo.

#Notamos que la variable "II3_1" aparece duplicada, así que nos quedamos con 1 sola:
column_values = X_nr.iloc[:, 115].values
X_nr["prueba"]=column_values
column_values_train = X_train.iloc[:, 115].values
X_train["prueba"]=column_values_train
X_nr=X_nr.drop(columns=["II3_1"])
X_train=X_train.drop(columns=["II3_1"])
X_nr = X_nr.rename(columns={'prueba': 'II3_1'})
X_train = X_train.rename(columns={'prueba': 'II3_1'})

#Usamos el modelo de vecinos cercanos con 3 vecinos para predecir la pobreza en la base norespondieron
knn = KNeighborsClassifier(n_neighbors=3)
knnfit= knn.fit(X_train, y_train)
y_pred_knn = knnfit.predict(X_nr)




In [ ]:
norespondieron['prediccion_pobreza'] = y_pred_knn

In [ ]:
#Nos quedamos con una observación por hogar para calcular la tasa de pobreza:

hogares = norespondieron.groupby(['CODUSU', 'NRO_HOGAR']).head(1)
pobres_count=(hogares['prediccion_pobreza'] == 1).sum()
nopobres_count=(hogares['prediccion_pobreza'] == 0).sum()
total=nopobres_count + pobres_count

print("El porcentaje de hogares pobres predicho en la muestra norespondieron es:", pobres_count*100/total)

El porcentaje de pobreza predicho es alto (72,37%). Podría ocurrir que las personas que no reportan su ingreso, o que reportan que es 0 sea porque este es bajo, en cuyo caso sería esperable que la tasa de pobreza en esta submuestra sea mayor que para el subconjunto de hogares que sí reportaron ingresos.